# COGS 108 - Data Checkpoint

# Names

- Jared (Ruotian) Chen
- Jimin Cheon
- Kane Gu
- Laurence D'Ercole
- Nisha Davankar

<a id='research_question'></a>
# Research Question

## YouTube
What attributes of a YouTube video affect its popularity/shareability (likes and/or shares)?

# Dataset(s)

## Dataset Information

- Dataset #1
    - Dataset Name: YouTube's [official data API](https://developers.google.com/youtube/v3) (video listing)
    - Link to the dataset: https://developers.google.com/youtube/v3/docs/videos/list
    - Number of observations: approx. `200 * 30 == 6000`

- Dataset #2
    - Dataset Name: YouTube's [official data API](https://developers.google.com/youtube/v3) (video searching)
    - Link to the dataset: https://developers.google.com/youtube/v3/docs/search/list
    - Number of observations: approx. `4000 * 30 == 120000` (subject to change)

- Dataset #3
    - Dataset Name: YouTube's internal video metadata API (specifically `window.ytInitialPlayerResponse`)
    - Link to the dataset: https://www.youtube.com/watch?v=[YOUTUBE_VIDEO_ID]
    - Number of observations: depending on the number of existing observations


Dataset #1 to #2 are used for retrieving information on likes/views/shares, genre, duration, date/time posted, content creator (i.e. subscription count), number of hashtags (from poster); Dataset #3 is specifically used for getting the number of ads in a video.

Duplicates in the datasets can be removed by using videos' unique IDs. Data cleaning and categorization happens in real time. Invalid or irrelevant entries, if any, will be immediately dropped. This design is intentional as we want the datasets to take as little space as possible (in memory and on disk).

# Setup

See the `EDA/` folder of this repo for details.

# Data Cleaning

Describe your data cleaning steps here.

Standard API responses typically require no data cleaning. Only basic encoding/decoding is required. See `EDA/{main,webapi,utils/decode}.py` for details. 

# Project Proposal (updated)

| Meeting Date | Meeting Time    | Completed Before Meeting                | Discuss at Meeting                            |
|--------------|-----------------|-----------------------------------------|-----------------------------------------------|
| 2/2          | 6:00 PM         | Project Proposal done                   | Roles, data wrangling                         |
| 2/9          | 6:00 PM         | Review feedback, Part of data wrangling | EDA ideas                                     |
| 2/16         | 6:00 PM         | Data wrangling and Partial EDA          | Discuss analysis and EDA                      |
| 3/2          | 6:00 PM         | Outline of Analysis and EDA             | Continue Analysis                             |
| 3/9          | 6:00 PM         | Finish Analysis                         | Ethics, Meeting with TA                       |
| 3/16         | 6:00 PM         | Draft done                              | Final review                                  |
| 3/19         | Before 11:59 PM | N/A                                     | Turn in Final Project & Group Project Surveys |